## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-27-19-02-52 +0000,bbc,Pubs given support package after business rate...,https://www.bbc.com/news/articles/c78vqj99168o...
1,2026-01-27-19-00-00 +0000,wsj,Philip Glass Withdraws Symphony From Kennedy C...,https://www.wsj.com/arts-culture/music/philip-...
2,2026-01-27-18-55-11 +0000,nypost,Several homes evacuated after reports of gunfi...,https://nypost.com/2026/01/27/us-news/sacramen...
3,2026-01-27-18-55-00 +0000,wsj,Work to Stop on $16 Billion Tunnel Project Unl...,https://www.wsj.com/us-news/work-to-stop-on-16...
4,2026-01-27-18-53-41 +0000,wapo,"Federal investigators say missed warnings, dan...",https://www.washingtonpost.com/transportation/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2457/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,67
147,ice,25
166,minnesota,22
215,minneapolis,21
150,pretti,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
147,2026-01-27-14-15-00 +0000,wsj,President Trump’s pivot in Minneapolis came af...,https://www.wsj.com/politics/policy/the-48-hou...,145
27,2026-01-27-18-14-21 +0000,wapo,Trump sends border czar to Minneapolis as Alex...,https://www.washingtonpost.com/nation/2026/01/...,143
258,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...,142
212,2026-01-27-10-28-15 +0000,cbc,Judge in Minnesota orders acting ICE director ...,https://www.cbc.ca/news/world/bovino-ice-minne...,142
184,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
147,145,2026-01-27-14-15-00 +0000,wsj,President Trump’s pivot in Minneapolis came af...,https://www.wsj.com/politics/policy/the-48-hou...
74,91,2026-01-27-17-12-56 +0000,nypost,"Alex Pretti was ‘known’ to feds, and had rib b...",https://nypost.com/2026/01/27/us-news/alex-pre...
258,67,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...
282,64,2026-01-27-00-42-00 +0000,wsj,The Army will gain access to Salesforce’s tech...,https://www.wsj.com/politics/national-security...
70,60,2026-01-27-17-20-02 +0000,nypost,"India, EU reach landmark trade deal to slash t...",https://nypost.com/2026/01/27/world-news/india...
253,54,2026-01-27-03-46-00 +0000,wsj,Democrats want changes to the Department of Ho...,https://www.wsj.com/politics/policy/government...
21,46,2026-01-27-18-23-07 +0000,nypost,10 found dead in NYC since start of Arctic win...,https://nypost.com/2026/01/27/us-news/10-found...
10,45,2026-01-27-18-41-00 +0000,wsj,Eleanor Holmes Norton said she won’t run for r...,https://www.wsj.com/politics/eleanor-holmes-no...
29,42,2026-01-27-18-09-32 +0000,latimes,Is social media harmful for kids? Meta and You...,https://www.latimes.com/california/story/2026-...
151,39,2026-01-27-14-05-17 +0000,bbc,'Crying horse' toys go viral in China ahead of...,https://www.bbc.com/news/articles/c70l7zndy3jo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
